In [1]:
import json
import pandas as pd
import os
import openpyxl
import numpy as np

In [2]:
with open("/home/phongnl/exam_DEA/json_file/1.json", "r") as f:
    data_dict = json.load(f)

In [3]:
data_dict['pageProps']['questions'][2]

{'question_id': 3,
 'choices': {'A': 'Use API calls to access and integrate third-party datasets from AWS Data Exchange.',
  'C': 'Use Amazon Kinesis Data Streams to access and integrate third-party datasets from AWS CodeCommit repositories.',
  'D': 'Use Amazon Kinesis Data Streams to access and integrate third-party datasets from Amazon Elastic Container Registry (Amazon ECR).',
  'B': 'Use API calls to access and integrate third-party datasets from AWS DataSync.'},
 'question_text': "A media company wants to improve a system that recommends media content to customer based on user behavior and preferences. To improve the recommendation system, the company needs to incorporate insights from third-party datasets into the company's existing analytics platform./n/nThe company wants to minimize the effort and time required to incorporate third-party datasets./n/nWhich solution will meet these requirements with the LEAST operational overhead?",
 'answer': 'A'}

In [4]:
list_questions = []
# for i in range(1,43):
for i in range(1,3):
    with open(rf"/home/phongnl/exam_DEA/json_file/{i}.json", "r") as f:
        data_dict = json.load(f)

    data_dict['pageProps']['questions'][0]

    questions = data_dict['pageProps']['questions']

    for question in questions:
        # Question text
        question_text = question['question_text']
        # print(question_text)
        # print(type(question_text))

        # Choices
        choices = question['choices']
        # for key, value in choices.items():
        #     print(f"{key}. {value}")
        # print(choices)
        # print(type(choices))

        # Correct answer
        correct_answer = question['answer']
        # print(f"Correct Answer: {correct_answer}")
        # print(type(correct_answer))

        # exam_id = question['exam_id']

        row = {
            "question_text": question_text,
            # "choices": choices,
            "correct_answer": correct_answer,
            # "exam_id": exam_id
        }

        question_df = pd.DataFrame([row])
        for key, val in choices.items():
            question_df[key] = val
        list_questions.append(question_df)
    # break

df = pd.concat(list_questions, ignore_index=True)
# df.to_excel("aws_dea_questions.xlsx", index=False)
# df.to_csv("aws_dea_questions.csv", index=False)

In [36]:
type1 = df[df['correct_answer'].str.len() == 1]
type2 = df[df['correct_answer'].str.len() > 1]

type1_split = np.array_split(type1, 9)
type2_split = np.array_split(type2, 9)

file_order = [0] * len(df)
for i in range(9):
    idx_type1 = type1_split[i].index.tolist()
    idx_type2 = type2_split[i].index.tolist()
    for idx in idx_type1 + idx_type2:
        file_order[idx] = i + 1

df['exam'] = file_order

/home/phongnl/exam_DEA/uv-project/.venv/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [6]:
df.to_json("aws_dea_questions.json", orient="records", lines=True)

In [39]:
df.to_csv("aws_dea_questions_final.csv", index=False)

In [7]:
df['exam_id'].value_counts()

exam_id
21    202
Name: count, dtype: int64

In [27]:
len(df[df['correct_answer'].str.len() > 1])

30

# Convert to json

In [7]:
df

,question_text,correct_answer,A,C,D,B,E
0,A data engineer is configuring an AWS Glue job...,D,Update the AWS Glue security group to allow in...,Review the AWS Glue job code to ensure that th...,Verify that the VPC's route table includes inb...,Configure an S3 bucket policy to explicitly gr...,NaN
1,A retail company has a customer data hub in an...,B,Create a separate table for each country's cus...,Move the data to AWS Regions that are close to...,Load the data into Amazon Redshift. Create a v...,Register the S3 bucket as a data lake location...,NaN
2,A media company wants to improve a system that...,A,Use API calls to access and integrate third-pa...,Use Amazon Kinesis Data Streams to access and ...,Use Amazon Kinesis Data Streams to access and ...,Use API calls to access and integrate third-pa...,NaN
3,A financial company wants to implement a data ...,BE,Use Amazon Aurora for data storage. Use an Ama...,Use AWS Glue DataBrew for centralized data gov...,Use Amazon RDS for data storage. Use Amazon EM...,Use Amazon S3 for data storage. Use Amazon Ath...,Use AWS Lake Formation for centralized data go...
4,A data engineer maintains custom Python script...,B,Store a pointer to the custom Python scripts i...,Store a pointer to the custom Python scripts i...,Assign the same alias to each Lambda function....,Package the custom Python scripts into Lambda ...,NaN
5,A data engineer is building a data orchestrati...,C,AWS Data Exchange,Amazon Managed Workflows for Apache Airflow (A...,AWS Glue,Amazon Simple Workflow Service (Amazon SWF),NaN
6,A gaming company uses a NoSQL database to stor...,C,Amazon Keyspaces (for Apache Cassandra),Amazon DynamoDB,Amazon Timestream,Amazon DocumentDB (with MongoDB compatibility),NaN
7,A data engineer creates an AWS Lambda function...,B,Ensure that the trust policy of the Lambda fun...,Ensure that the subnet where the Lambda functi...,Ensure that EventBridge schemas are valid and ...,Ensure that both the IAM role that EventBridge...,NaN
8,A company uses a data lake that is based on an...,B,Use both server-side encryption with AWS KMS k...,Use server-side encryption with customer-provi...,Use server-side encryption with AWS KMS keys (...,Use dual-layer server-side encryption with AWS...,NaN
9,A data engineer notices that Amazon Athena que...,B,Increase the query result limit.,Use federated queries.,Allow users who run the Athena queries to an e...,Configure provisioned capacity for an existing...,NaN


In [8]:
questions_list = []
for _, row in df.iterrows():
    # Lấy tất cả các cột chứa đáp án (A, B, C, D, E,...)
    options_cols = [col for col in row.index if col not in ['question_text', 'correct_answer', 'exam']]
    
    question_dict = {
        "question": row['question_text'],
        "options": [row[col] for col in options_cols if pd.notnull(row[col])],
        "answer": row['correct_answer'] if len(row['correct_answer']) == 1 else list(row['correct_answer']),
        "type": "single" if len(row['correct_answer']) == 1 else "multiple"
    }
    questions_list.append(question_dict)

# Lưu vào file JSON
with open('questions_formatted.json', 'w', encoding='utf-8') as f:
    json.dump(questions_list, f, ensure_ascii=False, indent=2)